In [ ]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
import time
from scipy.sparse import csc_matrix, coo_matrix, diags
import scipy as sp
from tqdm import tqdm

In [ ]:
import seaserpent as ss

# reading data

In [ ]:
info = pd.read_csv('https://raw.githubusercontent.com/flyconnectome/flywire_annotations/main/supplemental_files/Supplemental_file1_neuron_annotations.tsv',
                   delimiter = '\t')
info.columns

In [ ]:
info.super_class.value_counts(dropna = False)

In [ ]:
# to make the matrix smaller, let's take visual projection neurons as visual input, and remove the optic lobe neurons
cb = info[~info.super_class.isin(['optic'])]
# filter out the visual sensory neurons, except for the ocellar ones
cb = cb[(cb.cell_class != 'visual') | (cb.cell_type == 'ocellar retinula cell')]
cb

In [ ]:
# which sensory modalities are there?
cb[cb.super_class == 'sensory']['cell_class'].value_counts()

In [ ]:
cb["combined_type"] = cb.cell_type
# fill the leftover nas with hemibrain_type
cb.combined_type.fillna(cb.hemibrain_type, inplace= True)
cb.combined_type.value_counts()

## no type

In [ ]:
# how many neurons have no type information?
cb[['cell_type','hemibrain_type']].isna().value_counts()

In [ ]:
cbnotype = cb[cb.hemibrain_type.isna() & cb.cell_type.isna()]
cbnotype.status.value_counts(dropna=False)

In [ ]:
# which hemilineages do the neurons without a type belong to?
cbnotype[cbnotype.status.isna()].ito_lee_hemilineage.value_counts(dropna = False)

In [ ]:
cbnotype[cbnotype.status.isna()].super_class.value_counts()

In [ ]:
cbnotype.root_id[cbnotype.ito_lee_hemilineage.isna() & (cbnotype.super_class == 'central')].values

# Add CB types from live info?

In [ ]:
# read from info
infolive = ss.Table('info', base='main')
infodf = infolive.to_frame()
# remove not a neurons and duplicate entries
infodf = infodf[~infodf.super_class.isin(['not_a_neuron']) & ~infodf.status.isin(['bad_nucleus','not_a_neuron'])]

In [ ]:
infodfnotype = infodf[infodf.root_783.isin(cbnotype.root_id.astype(str))]
infodfnotype.cb_type.value_counts()

In [ ]:
# have a look at a few
infodfnotype[infodfnotype.cb_type == 'AN_6_22,29,36,37'].root_783.astype(np.int64).values

In [ ]:
# exclude ones with 'None' in the type name - they don't seem good
# first exclude the ones without a cb_type
infodfnotype = infodfnotype[~infodfnotype.cb_type.isna()]
infodfnotype = infodfnotype[['None' not in cbtype for cbtype in infodfnotype.cb_type]]
infodfnotype.cb_type.value_counts()

In [ ]:
infodfnotype.shape

So we can add additional labels to ~1700 neurons.

In [ ]:
cbtypes = dict(zip(infodfnotype.root_783.astype(np.int64), infodfnotype.cb_type))

In [ ]:
# fill the still leftover nas with cbtype
cb.combined_type.fillna(cb.root_id.map(cbtypes), inplace= True)

# Combine types into one column

In [ ]:
# fill the still leftover nas with root ids
cb.combined_type.fillna(cb.root_id.astype(str), inplace= True)

In [ ]:
typedict = dict(zip(cb.root_id, cb.combined_type))
sidedict = dict(zip(cb.root_id, cb.side))
type_super_class_dict = dict(zip(cb.combined_type, cb.super_class))
type_class_dict = dict(zip(cb.combined_type, cb.cell_class)) 
type_sub_class_dict = dict(zip(cb.combined_type, cb.cell_sub_class))

In [ ]:
import platform 

if platform.system() == 'Windows': 
    conn = pd.read_feather('C:/Users/44745/Downloads/syn_proof_analysis_filtered_consolidated_783.feather')
else: 
    conn = pd.read_feather('/Users/yijieyin/Downloads/syn_proof_analysis_filtered_consolidated_783.feather')

In [ ]:
# filter for the connectivity between components of the central brain
conn = conn[conn.pre_pt_root_id.isin(cb.root_id) & conn.post_pt_root_id.isin(cb.root_id)]

# add meta information
conn['pre_type'] = conn.pre_pt_root_id.map(typedict)
conn['pre_side'] = conn.pre_pt_root_id.map(sidedict)
conn['post_type'] = conn.post_pt_root_id.map(typedict)
conn['post_side'] = conn.post_pt_root_id.map(sidedict)

# and put type and side info into one column
# this separates the neurons of the same type from different sides, and
# keeps the possibility of looking at circuits where the side of the neuron makes a difference
conn['pre_type_side'] = conn.pre_type +'_'+ conn.pre_side
conn['post_type_side'] = conn.post_type +'_'+ conn.post_side
conn

In [ ]:
root_typeside_dict = dict(zip(conn.pre_pt_root_id, conn.pre_type_side))
root_typeside_dict.update(dict(zip(conn.post_pt_root_id, conn.post_type_side)))

type_typeside_dict = dict(zip(conn.pre_type_side, conn.pre_type))
type_typeside_dict.update(dict(zip(conn.post_type_side, conn.post_type)))
len(type_typeside_dict)

In [ ]:
# sum the number of synapses across neuropils and root_ids of the same type and side
conntt = conn.groupby(['pre_type_side','post_type_side','pre_type','post_type'])['syn_count'].sum().reset_index()
conntt

In [ ]:
# how many neurons are there roughly?
print('Total number of unique ids: ', len(set(conn.pre_pt_root_id)))
print('Total number of unique types (including where root_ids are used as a type): ', len(set(cb.combined_type)))
print('Total number of unique type_sides: ', len(set(conn.pre_type_side)))

In [ ]:
# the amount of memory it would take for a dense matrix in Gb
# number of integers * 64 bit integers / 8 bits per byte / 1e9 bytes per Gb
(len(set(conn.pre_type_side))**2)*64/8/1e9

- COO Matrix (Coordinate format):
1. In COO format, a sparse matrix is represented by three arrays: row indices, column indices, and the values themselves.  
2. This format is ideal for constructing sparse matrices when you have the coordinates of the non-zero elements (like from an edge list).  
3. It's efficient for matrix construction but not for matrix operations (like multiplication or indexing).  

- CSC Matrix (Compressed Sparse Column format):
1. In CSC format, the matrix is represented by column pointers, row indices, and the non-zero values.  
2. This format is efficient for arithmetic operations, column slicing, and matrix-vector products.  
3. However, constructing a CSC matrix directly from an edge list isn't as straightforward as with a COO matrix.

In [ ]:
# instead of making a dense matrix based on the edgelist above, let's make a sparse one from the edgelist directly
# first make a coo matrix
nodes = set(conntt.pre_type_side).union(set(conntt.post_type_side))
sorted_nodes = sorted(nodes)  # Convert the set to a sorted list
nodes_to_idx = {node:num for num, node in enumerate(sorted_nodes)}

# type to type connectivity
conntt['pre_idx'] = conntt['pre_type_side'].map(nodes_to_idx)
conntt['post_idx'] = conntt['post_type_side'].map(nodes_to_idx)

# Create COO matrix
row = conntt['pre_idx'].values
col = conntt['post_idx'].values
data = conntt['syn_count'].values
matrix_size = len(nodes)
coo = coo_matrix((data, (row, col)), shape=(matrix_size, matrix_size))

# then turn it into csc matrix
csc = coo.tocsc()

csc_size = csc.data.nbytes  # Size of the data array
csc_size += csc.indices.nbytes  # Size of the indices array
csc_size += csc.indptr.nbytes  # Size of the index pointer array
# number of MB
csc_size/1e6

In [ ]:
csc.shape

In [ ]:
csc

In [ ]:
col_sums = csc.sum(axis=0)
# Handling division by zero in case some columns have a sum of zero
# that is, where a neuron doesn't have incoming synapses
# .A turns it from a sparse matrix into a dense np array
col_sums_with_inversion = np.reciprocal(col_sums.A.squeeze().astype(float), where=col_sums.A.squeeze() != 0)
# Multiply each column by the inverse of its sum
inprop = csc.multiply(col_sums_with_inversion)
# and then reduce the precision to float32 to save memory
inprop = inprop.astype(np.float32)

In [ ]:
sp.sparse.save_npz('data/adult_inprop.npz', inprop)

In [ ]:
# save to csv, one row is one neuron 
meta = pd.DataFrame(root_typeside_dict.items(), columns = ['root_id','cell_type_side'])
meta['idx'] = meta.cell_type_side.map(nodes_to_idx)
meta['cell_type'] = meta.cell_type_side.map(type_typeside_dict)
meta['super_class'] = meta.cell_type.map(type_super_class_dict)
meta['cell_class'] = meta.cell_type.map(type_class_dict)
meta['cell_sub_class'] = meta.cell_type.map(type_sub_class_dict)
meta.to_csv('data/adult_type_meta.csv')

In [ ]:
# save the meta data as a csv, one row is one type_side  
meta = pd.DataFrame(nodes_to_idx.items(), columns=['cell_type_side','idx'])
meta['cell_type'] = meta.cell_type_side.map(type_typeside_dict)
meta['super_class'] = meta.cell_type.map(type_super_class_dict)
meta['cell_class'] = meta.cell_type.map(type_class_dict)
meta['cell_sub_class'] = meta.cell_type.map(type_sub_class_dict)
meta.to_csv('data/adult_type_meta.csv')

# NT

In [ ]:
cb.top_nt.value_counts(dropna = False)

In [ ]:
# first check consistency of NT within each type 
cb['top_nt'] = cb['top_nt'].fillna('unknown')
nt_count_per_type = cb.groupby('combined_type')['top_nt'].nunique()
plt.hist(nt_count_per_type)

In [ ]:
# For those with different NT for each neuron, get the majority NT for each type
nt_conflict_types = nt_count_per_type[nt_count_per_type > 1].index

# Getting counts of top_nt for each combined_type
nt_conflict_type_counts = cb[cb.combined_type.isin(nt_conflict_types)].groupby(['combined_type', 'top_nt']).size()

# Convert the series to a DataFrame and reset index
nt_conflict_type_counts = nt_conflict_type_counts.reset_index(name='counts')

# Sort by combined_type and counts in descending order
nt_conflict_type_counts.sort_values(by=['combined_type', 'counts'], ascending=[True, False], inplace=True)

# Initialize dictionary with existing type-nt matching
type_nt = dict(zip(cb.combined_type, cb.top_nt))
# Initialize a list to keep track of types with equal top_nt counts
types_with_equal_top_nt_counts = []

# Custom function to handle ties, random selection, and record keeping
def select_random_nt_and_record_ties(df):
    max_count = df['counts'].max()
    top_nts = df[df['counts'] == max_count]
    if len(top_nts) > 1:  # If there are ties
        types_with_equal_top_nt_counts.append(df['combined_type'].iloc[0])  # Record the type with ties
        return np.random.choice(top_nts['top_nt'].values)  # Random selection among ties
    else:
        return top_nts['top_nt'].values[0]

# Loop through each type to get the top_nt, handling ties appropriately
for atype in nt_conflict_types:
    type_df = nt_conflict_type_counts[nt_conflict_type_counts['combined_type'] == atype]
    top_nt = select_random_nt_and_record_ties(type_df)
    type_nt[atype] = top_nt

len(types_with_equal_top_nt_counts)

In [ ]:
# the types without any known nt 
[atype for atype, nt in type_nt.items() if nt == 'unknown']

In [ ]:
# now make a sparse matrix, where excitatory connections are 1, and inhibitory are -1 
# for now regarding everything (ACh, Glu, GABA, Dopamine, Serotonine, ??) except for Glu and GABA to be excitatory
# first make a dictionary 
nt_value = {'acetylcholine': 1, 
           'glutamate': -1, 
           'gaba': -1, 
           'dopamine': 1, 
           'serotonin': 1, 
           'octopamine': 1, 
           'unknown': 1}

type_sign = pd.DataFrame(type_nt.items(), columns = ['cell_type','nt'])
type_sign['sign'] = type_sign.nt.map(nt_value)
type_sign.to_csv('data/adult_nt.csv')

# int64 vs. int32

In [ ]:
# sensitivity analysis - does int32/16 matter?
import numpy as np
from scipy.sparse import random as sparse_random, csc_matrix

# Example matrix size
n = 100

# Generate a random sparse matrix (for demonstration purposes)
rng = np.random.default_rng()
matrix_float64 = sparse_random(n, n, density=0.1, format='csc', data_rvs=rng.random).astype(np.float64)
matrix_float32 = matrix_float64.astype(np.float32)
matrix_float16 = matrix_float64.astype(np.float16)

# Function to perform a series of matrix multiplications
def multiply_matrix(matrix, steps):
    result = matrix
    for _ in range(steps):
        result = result @ matrix
    return result

# Perform the operations
steps = 5
result_64 = multiply_matrix(matrix_float64, steps)
result_32 = multiply_matrix(matrix_float32, steps)
result_16 = multiply_matrix(matrix_float16, steps)

# Compare the results
diff_64_32 = np.abs(result_64 - result_32.astype(np.float64))
diff_64_16 = np.abs(result_64 - result_16.astype(np.float64))

# Print maximum differences
print(f"Max difference between float64 and float32: {diff_64_32.max()}")
print(f"Max difference between float64 and float16: {diff_64_16.max()}")

So we need float32.

# parallelising vs. not

In [ ]:
import multiprocessing
import time

import numpy as np

In [ ]:
num_cores = multiprocessing.cpu_count()

In [ ]:
# Define the matrix multiplication function
def matrix_multiply(args):
    A, B = args
    return np.dot(A, B)

In [ ]:
# Create two random matrices of size 1000x1000
A = np.random.rand(1000, 1000)
B = np.random.rand(1000, 1000)

# Split the matrices into 4 parts
A_parts = np.array_split(A, 4, axis=1)
B_parts = np.array_split(B, 4)

In [ ]:
from concurrent.futures import ProcessPoolExecutor


In [ ]:
start = time.time()

# # Map the matrix multiplication function to the 4 parts of the matrices
# C_parts = pool.map(matrix_multiply,
#                    [(A_part, B_part) for A_part, B_part in zip(A_parts, B_parts)])
with ProcessPoolExecutor() as executor:
    futures = [executor.submit(matrix_multiply, A_part, B_part) for A_part, B_part in zip(A_parts, B_parts)]
    C_parts = [f.result() for f in futures]

# Concatenate the parts of the result matrix
C = np.concatenate(C_parts, axis=1)

end = time.time()
print(end - start)